In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from matplotlib.backends.backend_pdf import PdfPages

# Function to plot histogram and save to PDF
def plot_histogram(chlorA_data, value_range, pdf):
    chlorA_data = chlorA_data[(chlorA_data >= value_range[0]) & (chlorA_data <= value_range[1])]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    bins = np.linspace(value_range[0], value_range[1], 11)
    hist, bins, patches = ax.hist(chlorA_data, bins=bins, edgecolor='black', color='blue')
    
    ax.set_xticks(bins)
    
    y_max = max(hist)
    print(y_max)
    a = 10**(len(str(int(y_max)))-1)
    arr = np.array([])
    for i in range(11):
        arr = np.append(arr, a*i)
    
    ax.set_yticks(arr)
    
    ax.set_xlabel('Chlorophyll-a Values')
    ax.set_ylabel('Frequency')
    ax.set_title('Histogram of Chlorophyll-a Data')
    
    pdf.savefig(fig)
    plt.close(fig)

# Function to plot CDF and save to PDF
def plot_cdf(value_range, chlorA_data, pdf):
    filtered_data = chlorA_data[(chlorA_data >= value_range[0]) & (chlorA_data <= value_range[1])]
    
    sorted_data = np.sort(filtered_data)
    
    cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(sorted_data, cdf, marker='.', linestyle='none')
    
    ax.set_xlabel('Chlorophyll-a Values')
    ax.set_ylabel('CDF')
    ax.set_title('CDF of Chlorophyll-a Data')
    
    ax.grid(True)
    
    pdf.savefig(fig)
    plt.close(fig)

# Open file in read mode
nc_file = nc.Dataset('AQUA_MODIS.20020101_20021231.L3m.YR.CHL.chlor_a.4km.nc', 'r')

chlorA_data = nc_file.variables['chlor_a'][:]
fill_value = -32767.0
chlorA_data = chlorA_data.flatten()
chlorA_data = chlorA_data[:]
regional_pixels = chlorA_data.shape[0]
chlorA_data = np.where(~np.isnan(chlorA_data), chlorA_data, np.nan)
chlorA_data = chlorA_data.flatten()
chlorA_data = chlorA_data[chlorA_data != fill_value]

latitudes = nc_file.variables['lat'][:]
fill_value = -999.0
nc_file.close()
latitudes = np.where(~np.isnan(latitudes), latitudes, np.nan)
latitudes = latitudes.flatten()
latitudes = latitudes[latitudes != fill_value]

# Define latitude bands
latitude_bands = {
    "30N-30S": (latitudes >= -30) & (latitudes <= 30)
}

# Define value ranges
value_ranges = [
    (0, 1), (0, 0.5), (0, 0.25), (0, 0.1),
    (0.01, 1), (0.01, 0.5), (0.01, 0.25), (0.01, 0.1)
]

# Function to calculate statistical metrics
def calculate_metrics(chlorA_data):
    mean_value = np.mean(chlorA_data)
    std_dev = np.std(chlorA_data)
    percentiles = np.percentile(chlorA_data, [50, 80, 90])
    return mean_value, std_dev, percentiles

# Create PDF file to save plots
with PdfPages('output_plots1.pdf') as pdf:
    # Loop through latitude bands and value ranges
    for band_name, band_filter in latitude_bands.items():
        filtered_latitudes = latitudes[band_filter]
        filtered_values = chlorA_data[:]
        
        print(f"Latitudes: {filtered_latitudes}")
        
        for value_range in value_ranges:
            range_filter = (filtered_values >= value_range[0]) & (filtered_values <= value_range[1])
            filtered_data = filtered_values[range_filter]
            valid_pixels = filtered_data.shape[0]
            Fraction_Valid = valid_pixels / regional_pixels
            
            if len(filtered_data) > 0:
                mean_value, std_dev, percentiles = calculate_metrics(filtered_data)
                print(f"\nLatitude Band: {band_name}")
                print(f"\nValue Range: {value_range}")
                print(f"Fraction_Valid: {Fraction_Valid}")
                print(f"Mean: {mean_value}")
                print(f"Standard Deviation: {std_dev}")
                print(f"50th Percentile: {percentiles[0]}")
                print(f"80th Percentile: {percentiles[1]}")
                print(f"90th Percentile: {percentiles[2]}")
                print("max_frequency::  ")
                plot_histogram(chlorA_data, value_range, pdf)
                plot_cdf(value_range, chlorA_data, pdf)
            else:
                print(f"\nValue Range: {value_range}")
                print("No data in this range.")
